In [1]:
# Import libraries and packages for the project 

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs4
from time import sleep
import csv
import requests

print('- Finish importing packages')

- Finish importing packages


In [2]:
# Task 1: Login to Linkedin
DRIVER_PATH = '../chromedriver'
# Task 1.1: Open Chrome and Access Linkedin login site
driver = webdriver.Chrome(executable_path=DRIVER_PATH)
sleep(2)
url = 'https://www.linkedin.com/login'
driver.get(url)
print('- Finish initializing a driver')
sleep(2)

# Task 1.2: Import username and password
credential = open('credentials.txt')
line = credential.readlines()
username = line[0]
password = line[1]
print('- Finish importing the login credentials')
sleep(1)

# Task 1.2: Key in login credentials
email_field = driver.find_element_by_id('username')
email_field.send_keys(username)
print('- Finish keying in email')
sleep(3)

password_field = driver.find_element_by_name('session_password')
password_field.send_keys(password)
print('- Finish keying in password')
sleep(3)

# Task 1.2: Click the Login button
signin_field = driver.find_element_by_xpath('//*[@id="organic-div"]/form/div[3]/button')
signin_field.click()
sleep(3)

print('- Finish Task 1: Login to Linkedin')




- Finish initializing a driver
- Finish importing the login credentials
- Finish keying in email
- Finish keying in password
- Finish Task 1: Login to Linkedin


In [3]:

# Task 2: Search for the profile we want to crawl

# Task 2.1: Locate the search bar element
# //*[@id="global-nav-typeahead"]/input
# 
search_field = driver.find_element_by_class_name('search-global-typeahead__input')

# Task 2.2: Input the search query to the search bar
keyword = 'Tester' + ' ' + 'people'
search_field.send_keys(keyword)

# Task 2.3: Search
search_field.send_keys(Keys.RETURN)


print('- Finish Task 2: Search for profiles')

- Finish Task 2: Search for profiles


In [4]:
sell_full_result = driver.find_element_by_xpath('//*[@id="main"]/div/div/div[1]/div[2]')
sell_full_result.click()

In [5]:
# Task 3: Scrape the URLs of the profiles

# Task 3.1: Write a function to extract the URLs of one page
def GetURL():
    page_source = bs4(driver.page_source)
    profiles = page_source.find_all('a', class_ = 'app-aware-link')
    all_profile_URL = []
    for profile in profiles:
        profile_ID = profile.get('href')
        profile_URL = profile_ID.split('?')[0]
        if profile_URL not in all_profile_URL:
            all_profile_URL.append(profile_URL)
    return all_profile_URL

# Task 3.2: Navigate through many page, and extract the profile URLs of each page

# input_page = int(input('How many pages you want to scrape: '))
input_page = 99
URLs_all_page = []
for page in range(input_page):

    URLs_one_page = GetURL()
    # print(URLs_one_page)
    sleep(2)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);') #scroll to the end of the page
    sleep(3)
    # next_button = driver.find_element_by_class_name("artdeco-pagination__button--next")
    try:
        next_button = driver.find_element_by_css_selector('[aria-label=Next]')
    except:
        print("Error when Loading")
        driver.refresh()
        continue
        
    driver.execute_script("arguments[0].click();", next_button)
    URLs_all_page = URLs_all_page + URLs_one_page
    sleep(3)

print('- Finish Task 3: Scrape the URLs')


import pandas as pd 
df = pd.DataFrame(URLs_all_page)
df.to_csv('profileURL_Tester.csv',mode='a', header=False)



- Finish Task 3: Scrape the URLs


In [18]:
driver.get(driver.getCurrentUrl());


AttributeError: 'WebDriver' object has no attribute 'getCurrentUrl'

In [7]:
# Task 4: Scrape the data of 1 Linkedin profile, and write the data to a .CSV file

with open('output.csv', 'w',  newline = '') as file_output:
    headers = ['Name', 'Job Title', 'Location', 'URL']
    writer = csv.DictWriter(file_output, delimiter=',', lineterminator='\n',fieldnames=headers)
    writer.writeheader()
    for linkedin_URL in URLs_all_page:
        driver.get(linkedin_URL)
        print('- Accessing profile: ', linkedin_URL)

        page_source = bs4(driver.page_source, "html.parser")

        info_div = page_source.find('div',{'class':'flex-1 mr5'})
        info_loc = info_div.find_all('ul')
        name = info_loc[0].find('li').get_text().strip() #Remove unnecessary characters 
        print('--- Profile name is: ', name)
        location = info_loc[1].find('li').get_text().strip() #Remove unnecessary characters 
        print('--- Profile location is: ', location)
        title = info_div.find('h2').get_text().strip()
        print('--- Profile title is: ', title)
        writer.writerow({headers[0]:name, headers[1]:location, headers[2]:title, headers[3]:linkedin_URL})
        print('\n')

print('Mission Completed!')

Mission Completed!


In [118]:
linkedin_URL ='https://www.linkedin.com/in/caroline-almaraz-942476168/'
driver.get(linkedin_URL)
html = driver.page_source
soup = bs4(html, "html.parser")

profile_experience = []
profile_education = []
profile_skills = []
profile_certification = []
profile_honors = []

# profile_name = soup.find_all('h1',{'class':'text-heading-xlarge inline t-24 v-align-middle break-words'})[0].get_text().strip()

profile_experience = soup.find_all('section',{'class':'artdeco-card'})
# print(len(profile_experience))
# print(soup)
# print(profile_name)
driver.execute_script('window.scrollTo(0, document.body.scrollHeight);') #scroll to the end of the page
for i in range(0,len(profile_experience)):
      header_tag = profile_experience[i].select('h2.pvs-header__title.text-heading-large>span.visually-hidden')
      if len(header_tag) > 0:
            if(header_tag[0].get_text().strip() == 'Experience'):
                  experience_list = profile_experience[i].select('div.display-flex.flex-row.justify-space-between')
                  print(experience_list)           
      
      # print(header_tag)
      # print(header_text)




[<div class="display-flex flex-row justify-space-between">
<div class="display-flex flex-column full-width">
<div class="display-flex align-items-center">
<span class="t-bold mr1">
<span aria-hidden="true"><!-- -->Customer service remote<!-- --></span><span class="visually-hidden"><!-- -->Customer service remote<!-- --></span>
</span>
<!-- --><!-- --><!-- --> </div>
<span class="t-14 t-normal">
<span aria-hidden="true"><!-- -->Creative Financial Staffing (CFS) · Full-time<!-- --></span><span class="visually-hidden"><!-- -->Creative Financial Staffing (CFS) · Full-time<!-- --></span>
</span>
<span class="t-14 t-normal t-black--light">
<span aria-hidden="true"><!-- -->Jan 2022 - Present · 3 mos<!-- --></span><span class="visually-hidden"><!-- -->Jan 2022 - Present · 3 mos<!-- --></span>
</span>
<span class="t-14 t-normal t-black--light">
<span aria-hidden="true"><!-- -->Ohio, United States<!-- --></span><span class="visually-hidden"><!-- -->Ohio, United States<!-- --></span>
</span>
</di